In [1]:
%matplotlib inline

import math
import pandas as pd
import joblib
import rec
from sklearn.model_selection import train_test_split

In [2]:
#Read the notifications table
notifications = pd.read_csv('https://raw.githubusercontent.com/pibeebeks/task-user-recommender/master/data/notifications.csv')
notifications.drop(['id', 'created_at', 'updated_at'], axis = 1, inplace=True)
notifications

,post_id,parent_comment_id,comment,sender_id,user_id,status,action,type
0,NaN,NaN,NaN,4,7,1,Followed,Following
1,NaN,NaN,NaN,1,4,1,Followed,Following
2,NaN,NaN,NaN,3,45,0,Followed,Following
3,NaN,NaN,NaN,1,4,1,Followed,Following
4,278.0,NaN,Hey bro!,1,4,1,Commented,Post
...,...,...,...,...,...,...,...,...
185,999.0,NaN,NaN,1,4,1,Like,Reaction
186,1007.0,NaN,NaN,3,3,0,Like,Reaction
187,998.0,NaN,NaN,2,2,0,Like,Reaction
188,999.0,NaN,NaN,4,4,1,Like,Reaction


In [3]:
train_data, test_data = train_test_split(notifications, test_size = 0.40, random_state=0)
print(train_data.head(5))

     post_id  parent_comment_id                  comment  sender_id  user_id  \
62       NaN                NaN                      NaN       2234        6   
2        NaN                NaN                      NaN          3       45   
59       NaN                NaN                      NaN       2234        6   
110      NaN                NaN                      NaN       2277        6   
178   1002.0                NaN  just some funny comment         22       22   

     status     action       type  
62        0   Followed  Following  
2         0   Followed  Following  
59        0   Followed  Following  
110       0   Followed  Following  
178       0  Commented       Post  


In [4]:
recommender = rec.popularity_recommender_py()
recommender.create(train_data, 'user_id')

/Users/nedurobert/Documents/SE/HNG/ML/task-user-recommender/rec.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['eventStrength'] = train_data['action'].apply(lambda x: event_type_strength[x]).copy()


In [5]:
user_id = 3
recommender.recommend(user_id)

/anaconda3/envs/ml/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_id
1,2
0,1
3,7


In [6]:
filename = 'popular.sav'
joblib.dump(recommender, filename)

['popular.sav']